In [ ]:
#Downloading the experiment files.
!gdown --id 1rLiYwui_vwmDttMrklbA4jKlixevpy4_
!gdown --id 1Tx0y0XsBP4OASnuTA7IhIQoCHTuSeIGR
!gdown --id 1YzyQnwBLZc-hTJ67ReQALOuYuq-vwsux

In [ ]:
!pip install iglovikov_helper_functions
!pip install people_segmentation  > /dev/null

In [ ]:
import numpy as np
import cv2
import torch
import albumentations as albu
from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
from people_segmentation.pre_trained_models import create_model
import matplotlib.pyplot as plt
%matplotlib inline

model = create_model("Unet_2020-07-20")
model.eval();
transform = albu.Compose([albu.Normalize(p=1)], p=1)

In [ ]:
image = load_rgb("/content/Robert-Downey.jpg")

In [ ]:
plt.imshow(image)

In [ ]:
padded_image, pads = pad(image, factor=32, border=cv2.BORDER_CONSTANT)
x = transform(image=padded_image)["image"]
x = torch.unsqueeze(tensor_from_rgb_image(x), 0)

with torch.no_grad():
  prediction = model(x)[0][0]

mask = (prediction > 0).cpu().numpy().astype(np.uint8)
mask = unpad(mask, pads)

In [ ]:
plt.imshow(mask,cmap='gray')

In [ ]:
print(model)

In [ ]:
black_img=np.full(mask.shape,0,dtype=np.uint8)
stacked_img = np.stack((black_img,)*3, axis=-1)

In [ ]:
stacked_img[mask==1]=image[mask==1]
plt.imshow(stacked_img)

In [ ]:
h,w=stacked_img.shape[:-1]
img2=cv2.cvtColor(cv2.imread('/content/landscape.jpg'),cv2.COLOR_BGR2RGB)
img2=cv2.resize(img2,(w,h))
stacked_img[mask!=1]=img2[mask!=1]
plt.imshow(stacked_img)

In [ ]:
def replace_background(img,background):
  padded_image, pads = pad(img, factor=32, border=cv2.BORDER_CONSTANT)
  x = transform(image=padded_image)["image"]
  x = torch.unsqueeze(tensor_from_rgb_image(x), 0)

  with torch.no_grad():
    prediction = model(x)[0][0]

  mask = (prediction > 0).cpu().numpy().astype(np.uint8)
  mask = unpad(mask, pads)

  black_img=np.full(mask.shape,0,dtype=np.uint8)
  stacked_img = np.stack((black_img,)*3, axis=-1)
  stacked_img[mask==1]=img[mask==1]

  h,w=stacked_img.shape[:-1]

  background=cv2.resize(background,(w,h))
  stacked_img[mask!=1]=background[mask!=1]

  return stacked_img

In [ ]:
#Video frame background replacement.
import cv2
  
background=cv2.cvtColor(cv2.imread('/content/landscape.jpg'),cv2.COLOR_BGR2RGB)
cam = cv2.VideoCapture('/content/agai_robert.mp4')
   
if (cam.isOpened() == False): 
    print("Error reading video file")
   
fourcc = cv2.VideoWriter_fourcc(*'XVID')
vidwriter = cv2.VideoWriter('filtered_video.avi',fourcc, 20.0, (1280,720))

# count=0
while(True):
    ret, frame = cam.read()
    if ret == True: 
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        r_img=replace_background(frame,background)
        r_img=cv2.cvtColor(r_img,cv2.COLOR_RGB2BGR)
        vidwriter.write(r_img)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break

    else:
        break
  
cam.release()
vidwriter.release()
    
cv2.destroyAllWindows() 
print("Sucessfully processed the video.")